## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [4]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gtts 2.5.4 requires click<8.2,>=7.1, but you have click 8.2.1 which is incompatible.


## Preparing your streaming agent for deployment on AgentCore Runtime

Let's now deploy our streaming agents to AgentCore Runtime. The streaming functionality is handled automatically by the AgentCore SDK when you use async generators or yield statements in your entrypoint function.

Key points for streaming implementation:
* Use `async def` for your entrypoint function
* Use `yield` to stream chunks as they become available
* The AgentCore SDK automatically handles the Server-Sent Events (SSE) format
* Clients will receive Content-Type: text/event-stream responses

### Strands Agents with Amazon Bedrock model and Streaming
Let's look at our streaming implementation for the Strands Agent using Amazon Bedrock model.

## Deploying the streaming agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial we will use the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [8]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_streaming.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_claude_streaming"
)
response

Entrypoint parsed: file=/home/ubuntu/workspace/robot-agentic-ai/be/strands_claude_streaming.py, bedrock_agentcore_name=strands_claude_streaming
Configuring BedrockAgentCore agent: strands_claude_streaming


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/ubuntu/workspace/robot-agentic-ai/be/Dockerfile
Generated .dockerignore: /home/ubuntu/workspace/robot-agentic-ai/be/.dockerignore
Keeping 'strands_claude_streaming' as default agent
Bedrock AgentCore configured: /home/ubuntu/workspace/robot-agentic-ai/be/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/ubuntu/workspace/robot-agentic-ai/be/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/workspace/robot-agentic-ai/be/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/workspace/robot-agentic-ai/be/.dockerignore'), runtime='Docker', region='us-west-2', account_id='533267442321', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching streaming agent to AgentCore Runtime

Now that we've got a docker file, let's launch the streaming agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [9]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_streaming' to account 533267442321 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_streaming
✅ ECR repository available: 533267442321.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming
Getting or creating execution role for agent: strands_claude_streaming
Using AWS region: us-west-2, account ID: 533267442321
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-2ce6205d24


✅ Reusing existing ECR repository: 533267442321.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming


✅ Reusing existing execution role: arn:aws:iam::533267442321:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-2ce6205d24
✅ Execution role available: arn:aws:iam::533267442321:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-2ce6205d24
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_streaming
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-2ce6205d24
Reusing existing CodeBuild execution role: arn:aws:iam::533267442321:role/AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-2ce6205d24
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_claude_streaming/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_claude_streaming-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.2s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.0s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLO

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [10]:
import time

status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_streaming


'READY'

### Invoking AgentCore Runtime with Streaming

Finally, we can invoke our AgentCore Runtime with a payload and receive streaming responses

<div style="text-align:left">
    <img src="images/invoke.png" width="85%"/>
</div>

In [12]:
import uuid

session_id = uuid.uuid4()
invoke_response = agentcore_runtime.invoke({
    "prompt": 
    "지금 날씨 어때?"
}, session_id=str(session_id))
invoke_response

Invoking BedrockAgentCore agent 'strands_claude_streaming' via cloud endpoint


현

재 날

씨 

정보를 

...

기

 좋은

 날이

네요!

{}

### Invoking AgentCore Runtime with boto3 for Streaming

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For streaming responses, you'll need to handle the Server-Sent Events format.

In [6]:
import boto3
import json
from IPython.display import Markdown, display

agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# For streaming responses, we need to handle the EventStream
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2+1"})
)

# Check if the response is streaming
if "text/event-stream" in boto3_response.get("contentType", ""):
    print("Processing streaming response with boto3:")
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            print(line)
            if line.startswith("data: "):
                data = line[6:].replace('"', '')  # Remove "data: " prefix
                print(f"Received streaming chunk: {data}")
                content.append(data.replace('"', ''))
    
    # Display the complete streamed response
    full_response = " ".join(content)
    display(Markdown(full_response))
else:
    # Handle non-streaming response
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    
    if events:
        try:
            response_data = json.loads(events[0].decode("utf-8"))
            display(Markdown(response_data))
        except:
            print(f"Raw response: {events[0]}")

Processing streaming response with boto3:
data: "I"
Received streaming chunk: I
data: "'ll"
Received streaming chunk: 'll
data: " calculate that"
Received streaming chunk:  calculate that
data: " for you using"
Received streaming chunk:  for you using
data: " the calculator"
Received streaming chunk:  the calculator
data: " function"
Received streaming chunk:  function
data: "."
Received streaming chunk: .
data: "The"
Received streaming chunk: The
data: " result of"
Received streaming chunk:  result of
data: " 2 +"
Received streaming chunk:  2 +
data: " 1 ="
Received streaming chunk:  1 =
data: " 3."
Received streaming chunk:  3.


I 'll  calculate that  for you using  the calculator  function . The  result of  2 +  1 =  3.

## Benefits of Streaming Responses

Streaming responses provide several key advantages:

### User Experience
* **Immediate Feedback**: Users see partial results as they become available
* **Perceived Performance**: Responses feel faster even if total time is the same
* **Progressive Display**: Long responses can be displayed incrementally

### Technical Benefits
* **Memory Efficient**: Process large responses without loading everything into memory
* **Timeout Prevention**: Avoid timeouts on long-running operations
* **Real-time Processing**: Handle real-time data as it becomes available

### Use Cases
* **Content Generation**: Long-form writing, reports, documentation
* **Data Analysis**: Progressive results from complex calculations
* **Multi-step Workflows**: Show progress through complex agent reasoning
* **Real-time Monitoring**: Live updates from monitoring agents

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [18]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('533267442321.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_claude_streaming',
 'strands_claude_streaming-VqDW8eF351',
 'bedrock-agentcore-strands_claude_streaming')

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!

You have successfully implemented and deployed a streaming agent using Amazon Bedrock AgentCore Runtime! 

## What you've learned:
* How to implement streaming responses using async generators
* How AgentCore Runtime automatically handles SSE format
* How to process streaming responses on the client side
* The benefits of streaming for user experience and performance

## Next steps:
* Experiment with different streaming patterns for your use cases
* Implement custom streaming logic for complex multi-step workflows
* Explore combining streaming with other AgentCore features like Memory and Gateway
* Consider implementing client-side streaming visualization for better UX

In [1]:
launch_result

NameError: name 'launch_result' is not defined

In [13]:
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
import base64
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime


# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time."""
)

In [17]:
user_input = "날씨 어때"
try:
    accumulated_response = ""
    async for event in agent.stream_async(user_input):
        print(event)
        if "data" in event:
            text_data = event["data"]
            accumulated_response += text_data
            if text_data.strip():
                print(f"data: {json.dumps({'type': 'content', 'content': text_data})}\n\n")
        elif "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"data: {json.dumps({'type': 'tool_use', 'tool_name': tool_name})}\n\n")
except Exception as e:
    print(f"data: {json.dumps({'type': 'content', 'content': f'Error: {str(e)}'})}\n\n")

{'init_event_loop': True}
{'start': True}
{'start_event_loop': True}
{'event': {'messageStart': {'role': 'assistant'}}}
{'event': {'contentBlockDelta': {'delta': {'text': '현'}, 'contentBlockIndex': 0}}}
현{'data': '현', 'delta': {'text': '현'}, 'agent': <strands.agent.agent.Agent object at 0x7dbc1fb80440>, 'event_loop_cycle_id': UUID('a69b0c68-2a06-4bc2-878a-acf340d45236'), 'request_state': {}, 'event_loop_cycle_trace': <strands.telemetry.metrics.Trace object at 0x7dbc1faa4ef0>, 'event_loop_cycle_span': NonRecordingSpan(SpanContext(trace_id=0x00000000000000000000000000000000, span_id=0x0000000000000000, trace_flags=0x00, trace_state=[], is_remote=False))}
data: {"type": "content", "content": "\ud604"}


{'event': {'contentBlockDelta': {'delta': {'text': '재 날'}, 'contentBlockIndex': 0}}}
재 날{'data': '재 날', 'delta': {'text': '재 날'}, 'agent': <strands.agent.agent.Agent object at 0x7dbc1fb80440>, 'event_loop_cycle_id': UUID('a69b0c68-2a06-4bc2-878a-acf340d45236'), 'request_state': {}, 'event_

In [ ]:
stream = agent.stream_async(user_message)
async for event in stream:            
    if "message" in event and "content" in event["message"] and "role" in event["message"] and event["message"]["role"] == "assistant":
        for content_item in event['message']['content']:
            if "toolUse" in content_item and "input" in content_item["toolUse"] and content_item["toolUse"]['name'] == 'execute_sql_query':
                print(f" {content_item['toolUse']['input']['description']}.\n\n")
            elif "toolUse" in content_item and "name" in content_item["toolUse"] and content_item["toolUse"]['name'] == 'get_tables_information':
                print("\n\n")
            elif "toolUse" in content_item and "name" in content_item["toolUse"] and content_item["toolUse"]['name'] == 'current_time':
                print("\n\n")
    elif "data" in event:
        yield event['data']